In [ ]:
# test (miniconda3 (Python 3.12.2), open integrated terminal)

In [1]:
import csv

In [3]:
close_prices = []

with open('./data/prices/csvs/XOM.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        close_prices.append(row[1])

print(close_prices)

ValueError: invalid literal for int() with base 10: '99.98'